# Ensemble Model Training and Evaluation

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score

# 1. Data Loading and Preparation
data = pd.read_csv('emotion_data.csv')

# Define the target variable and feature columns
target_col = 'predicted_emotion'
feature_cols = [
    'sentiment_score', 'text_length', 'word_count', 
    'avg_word_length', 'stopword_count', 
    'first_person_pronoun_count', 'keyword_count'
]

# Select features (X) and target (y)
X = data[feature_cols]
y = data[target_col]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 2. Splitting and Scaling the Data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Define Individual Models for the Ensemble
clf1 = LogisticRegression(max_iter=1000, random_state=42)
clf2 = SVC(probability=True, random_state=42)  # Enable probability estimates for soft voting
clf3 = RandomForestClassifier(random_state=42)

# 4. Build the Voting Ensemble (soft voting)
ensemble_clf = VotingClassifier(
    estimators=[('lr', clf1), ('svm', clf2), ('rf', clf3)],
    voting='soft'
)

# Train the ensemble model
ensemble_clf.fit(X_train_scaled, y_train)

# 5. Evaluate the Ensemble Model
y_pred = ensemble_clf.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Ensemble Model Accuracy: {:.4f}".format(accuracy))
print("Ensemble Model F1 Score: {:.4f}".format(f1))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Ensemble Model Accuracy: 0.7092
Ensemble Model F1 Score: 0.5913

Classification Report:
              precision    recall  f1-score   support

   Uncertain       0.71      1.00      0.83      6553
       anger       0.00      0.00      0.00       167
     disgust       0.00      0.00      0.00         9
        fear       0.00      0.00      0.00       668
         joy       0.00      0.00      0.00       232
     neutral       0.00      0.00      0.00        12
     sadness       0.34      0.01      0.01      1520
    surprise       0.00      0.00      0.00        69

    accuracy                           0.71      9230
   macro avg       0.13      0.13      0.11      9230
weighted avg       0.56      0.71      0.59      9230



C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


The ensemble aimed to tackle class imbalance and improve emotion detection. While it reached 71% accuracy, a weighted F1 of 0.59 and macro F1 of 0.11 reveal heavy bias toward the "Uncertain" class, with minority classes scoring 0. Individual models and neural networks performed better (F1: 0.70–0.75), indicating the need for resampling or class weighting to refine the ensemble.